In [ ]:
#

In [ ]:
import os

import numpy as np
import pandas as pd

import nibabel as nib
from nilearn.input_data import NiftiMasker
from nilearn.image import concat_imgs

In [ ]:
# Load an image from a single brain:
def img_load(folder, img):
    img_path = os.path.join(folder, img)
    fmri_image = nib.load(img_path)
    #print(fmri_image.shape)
    return fmri_image

In [ ]:
# Load a resampled, binarised GM mask
def mask_img_load(folder, img):
    img_path = os.path.join(folder, img)
    mask = nib.load(img_path)
    return mask

In [ ]:
# Extract the timeseries from a single session
def extract_time_series(fmri_image, mask):
    brain_masker = NiftiMasker(mask, memory='nilearn_cache', memory_level=1, verbose=0)
    brain_time_series = brain_masker.fit_transform(fmri_image)
    return brain_time_series, brain_masker

In [ ]:
# Transform the FC or DC array back into a NIfTI volume and save
def array_to_nifti(brain_masker,array):
    img = brain_masker.inverse_transform(array.T)
    return img

In [ ]:
# Get the ID's
os.chdir('/Users/mishodimitrov/Downloads/PhD/Analysis/CBD/Data/Covariates')
covariate_sheet = pd.read_excel('Covariates.xlsx', sheet_name=0, engine='openpyxl')

In [ ]:
#covariate_sheet = covariate_sheet.iloc[:103,:7]
covariate_sheet

In [ ]:
id_list = [covariate_sheet.iloc[:,0][i] for i in range(covariate_sheet.iloc[:,0].shape[0])]
id_list

In [ ]:
placebo_list = []
cbd_list = []

In [ ]:
# Split the sessions into the 2 respective lists from above
for i in range(covariate_sheet.shape[0]):
    if covariate_sheet.iloc[i]["Placebo"] == 1:
        placebo_list.append(covariate_sheet.iloc[i]["ID"])
    elif covariate_sheet.iloc[i]["CBD"] == 1:
        cbd_list.append(covariate_sheet.iloc[i]["ID"])
    else:
        print("Something's wrong here..")

In [ ]:
#cbd_list

In [ ]:
placebo_td = []
placebo_autism = []
cbd_td = []
cbd_autism = []

In [ ]:
# Split each of those into 2 new lists, of TD and ASC sessions, respectively (see just above)
for i in range(len(placebo_list)):
    if int(placebo_list[i][0]) == 0:
        placebo_td.append(placebo_list[i])
    elif int(placebo_list[i][0]) == 1:
        placebo_autism.append(placebo_list[i])
    else:
        print("Something's wrong here..")
        
for i in range(len(cbd_list)):
    if int(cbd_list[i][0]) == 0:
        cbd_td.append(cbd_list[i])
    elif int(cbd_list[i][0]) == 1:
        cbd_autism.append(cbd_list[i])
    else:
        print("Something's wrong here..")

In [ ]:
# Only placebo and CBD from the ASC group
print(placebo_autism)
print(cbd_autism)

In [ ]:
# Remove the ones that only have a single session
placebo_autism_to_remove = ['102A', '110C']
cbd_autism_to_remove = ['104A']
for i in placebo_autism_to_remove:
    print('Removing session ' + i)
    placebo_autism.remove(i)
for j in cbd_autism_to_remove:
    print('Removing session ' + j)
    cbd_autism.remove(j)

In [ ]:
print(len(placebo_td))
print(len(cbd_td))

In [ ]:
work_dir = '/Users/mishodimitrov/Downloads/PhD/Analysis/CBD/Static_FC/DC_data/wDC_Z/'

In [ ]:
ses_id = cbd_td

In [ ]:
project_name =  'MAGACAN'
name_ending = '_wDC_Z.nii'

In [ ]:
img_list = []
for i in range(len(ses_id)):
    session_filename = work_dir + project_name + ses_id[i] + name_ending
    img_list.append(session_filename)

In [ ]:
img_list

In [ ]:
# Concatenate all 3D images for the respective group into a 4D image and save
os.chdir('/Users/mishodimitrov/Downloads/PhD/Analysis/CBD/Static_FC/Analysis/Step_3_Voxel-wise_wDC')
group_img = concat_imgs(img_list)
nib.save(group_img, 'td_cbd_wDC_group_img')

In [ ]:
# Combine TD and ASC shift images
shift_img_list = ['/Users/mishodimitrov/Downloads/PhD/Analysis/CBD/Static_FC/Analysis/Step_3_Voxel-wise_wDC/td_shift_wDC_group_img.nii.gz',
                 '/Users/mishodimitrov/Downloads/PhD/Analysis/CBD/Static_FC/Analysis/Step_3_Voxel-wise_wDC/autism_shift_wDC_group_img.nii.gz']
shift_img = concat_imgs(shift_img_list)
nib.save(shift_img, 'shift_comp_group_img')

In [ ]:
asc_pla = concat_imgs(img_list[0:17])
asc_tia = concat_imgs(img_list[17:34])

In [ ]:
nib.save(asc_pla, 'ASC_placebo_wDC_group_img')
nib.save(asc_tia, 'ASC_tianeptine_wDC_group_img')

In [ ]:
td_pla = concat_imgs(img_list[0:19])
td_tia = concat_imgs(img_list[19:38])

In [ ]:
nib.save(td_pla, 'TD_placebo_wDC_group_img')
nib.save(td_tia, 'TD_tianeptine_wDC_group_img')